# Image Quality Quantitative Metrics - Inception Score (IS)

## Implementation & Results

The purpose of this notebook is to implement and collect the results of the inception score metric for quantitative evaluation of the synthetic image quality as outlined in sections 3.5.1 and 4.1.1 of the bachelor thesis.

The code provided in this notebook was developed using the Kaggle platform.

The Inception Score applied a Inception-V3 as a feature extractor. The code in this notebook incorporates the following sources as references:

- https://github.com/sbarratt/inception-score-pytorch
- https://pytorch.org/hub/pytorch_vision_inception_v3/



## Step 1 - Importing Dependencies

- Importing the necessary libraries to execute the code.

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.models.inception import Inception_V3_Weights
from torchvision.datasets import ImageFolder
import numpy as np
from scipy.stats import entropy

## Step 2 - Hyperparameter Settings

- Set the HPs for the Inception-V3 model. Besides, also check whether a GPU is available for use.

In [ ]:
batch_size = 32       # Batch size for the Inception-V3 network
splits = 10           # Number of splits of the data to perform the IS evaluation
cuda = True

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(f'Selected device: {device}')

## Step 3 - Dataset Loading

- Loading the dataset as a PyTorch dataset from the `ImageFolder` method.
- Defining the necessary transformations from the oficial PyTorch implementation: [Inception_v3 PyTorch](https://pytorch.org/hub/pytorch_vision_inception_v3/).

In [3]:
transforms = transforms.Compose([transforms.Resize(299),
                                 transforms.Grayscale(num_output_channels=3),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root='/path/to/images/root/folder', transform=transforms)

- Function to correct the dataset format used in the Inception-V3 model
- The funtion remove the labels tensor from the dataloader, returning a list of image tensors without labels.

In [5]:
def collate_fn(batch):
    return [x[0] for x in batch]

## Step 4 - Inception Score (IS) Evaluation

- The following function computes the Inception Score of the generated images defined in the dataset field on step 3.

In [6]:
def inception_score(imgs, cuda=cuda, batch_size=32, resize=False, splits=10):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size, collate_fn=collate_fn)

    # Load inception model
    inception_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', weights=Inception_V3_Weights.IMAGENET1K_V1).to(device)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = torch.stack(batch).type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


## Step 5 - Inception Score Results

- The code below computes the mean and std of the IS for the desired number of splits in the dataset previously defined.
- **OBS.:** The IS value for the raw training dataset should be consider the upper threshold of this analysis.

In [ ]:
is_mean, is_std = inception_score(dataset, cuda=cuda, batch_size=batch_size, resize=False, splits=splits)

print(f"Mean IS: {np.mean(is_mean)}")
print(f"Std IS: {np.std(is_std)}")